In [ ]:
import numpy as np
import pandas as pd
from sklearn.metrics import mean_absolute_error
from sklearn.model_selection import KFold, train_test_split, cross_val_score
from sklearn.kernel_ridge import KernelRidge
from hyperopt import fmin, tpe, hp, STATUS_OK, Trials
from KernelRidge_commutator import generate_CM, KRR_commutator
import matplotlib.pyplot as plt
import matplotlib.ticker as ticker
import warnings
import multiprocessing

## Load Data ##

In [ ]:
data = np.load('qm9_data.npz', allow_pickle=True)
coords = data['coordinates']
nuclear_charges = data['charges']
elements = data['elements']
energies = np.array(data['H_atomization'])

In [ ]:
num_mol = energies.shape[0]
cm = np.zeros((num_mol, 29, 29))
for i in range(num_mol):
    cm[i] = generate_CM(coords[i], nuclear_charges[i], pad=29)

print(cm.shape)

In [ ]:
keys = list(data.keys())
print(keys)

In [ ]:
print(elements[0])

## Model ##

In [ ]:
X = cm[:5000]
y = energies.reshape(-1, 1)[:5000]
params = {'lambda': 1e-3, 'length': 1}
mae_scores = []

kfold = KFold(n_splits=2, shuffle=True, random_state=42)
for fold, (train_index, test_index) in enumerate(kfold.split(X)):
    X_train, X_test = X[train_index], X[test_index]
    y_train, y_test = y[train_index], y[test_index]
    preds = KRR_commutator(X_train, y_train, X_test, params, kernel='rbf')
    score = mean_absolute_error(preds.reshape(-1, 1), y_test)
    mae_scores.append(score)
    print(f"fold {fold+1}: MAE = {score}")

print(f"Average MAE: {np.array(mae_scores).mean()}")


In [ ]:
X = cm[:2000]
y = energies.reshape(-1, 1)[:2000]

def objective(params):
    mae_scores = []
    kfold = KFold(n_splits=2, shuffle=True, random_state=42)
    for fold, (train_index, test_index) in enumerate(kfold.split(X)):
        X_train, X_test = X[train_index], X[test_index]
        y_train, y_test = y[train_index], y[test_index]
        preds = KRR_commutator(X_train, y_train, X_test, params, kernel='rbf')
        if type(preds) is str:
            return np.inf
        score = mean_absolute_error(preds.reshape(-1, 1), y_test)
        mae_scores.append(score)
    return np.array(mae_scores).mean()

space = {
    'lambda': hp.loguniform('lambda', -30, 0), 
    'length': hp.loguniform('length', -2, 2)
}

trials = Trials()
with warnings.catch_warnings():
    warnings.filterwarnings('ignore')
    best = fmin(fn=objective,
                space=space,
                algo=tpe.suggest, # tree parzen estimator
                max_evals=50,
                trials=trials)

print("Best hyperparameters:", best)
print("Loss:", trials.best_trial['result']['loss'])

In [ ]:
def objective(params):
    mae_scores = []
    kfold = KFold(n_splits=2, shuffle=True, random_state=42)
    for fold, (train_index, test_index) in enumerate(kfold.split(X)):
        X_train, X_test = X[train_index], X[test_index]
        y_train, y_test = y[train_index], y[test_index]
        preds = KRR_commutator(X_train, y_train, X_test, params, kernel='laplacian')
        if type(preds) is str:
            return np.inf
        score = mean_absolute_error(preds.reshape(-1, 1), y_test)
        mae_scores.append(score)
    return np.array(mae_scores).mean()

space = {
    'lambda': hp.loguniform('lambda', -30, 0), 
    'length': hp.loguniform('length', -2, 2)
}

trials = Trials()
with warnings.catch_warnings():
    warnings.filterwarnings('ignore')
    best = fmin(fn=objective,
                space=space,
                algo=tpe.suggest, # tree parzen estimator
                max_evals=50,
                trials=trials)

print("Best hyperparameters:", best)
print("Loss:", trials.best_trial['result']['loss'])

## Baseline ##

In [ ]:
cm_flatten = cm.reshape((cm.shape[0], -1))
print(cm_flatten.shape)

In [ ]:
X = cm_flatten[:2000]
y = energies.reshape(-1, 1)[:2000]

params = {'alpha': 1e-5, 'gamma': 1e-5, 'kernel': 'rbf'}
model = KernelRidge(**params)

neg_mae_scores = cross_val_score(model, X, y, scoring = 'neg_mean_absolute_error', cv=2)
mae_scores = -neg_mae_scores
print(mae_scores.mean())


## Learning Curve ##

In [ ]:
X = cm[:10000]
y = energies.reshape(-1, 1)[:10000]

gaussian_error = []
laplacian_error = []
training_size = [i*1000 for i in range(1, 10, 2)]

best_params_gaussian = {'lambda': 7.882813671669788e-12, 'length': 0.7310621690634572}
best_params_laplacian = {'lambda': 1.9168314543935938e-13, 'length': 6.657357327134051}

for size in training_size:
    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=1-size/10000, random_state=42)
    preds_gaussian = KRR_commutator(X_train, y_train, X_test, best_params_gaussian, kernel='rbf')
    preds_laplacian = KRR_commutator(X_train, y_train, X_test, best_params_laplacian, kernel='laplacian')
    score_gaussian = mean_absolute_error(preds_gaussian.reshape(-1, 1), y_test)
    score_laplacian = mean_absolute_error(preds_laplacian.reshape(-1, 1), y_test)
    gaussian_error.append(score_gaussian)
    laplacian_error.append(score_laplacian)
    print(f"Training size: {size}. Gaussian MAE: {score_gaussian}. Laplacian MAE: {score_laplacian}")

print(gaussian_error)
print(laplacian_error)

In [ ]:
X = cm_flatten[:10000]
y = energies.reshape(-1, 1)[:10000]

normal_KRR_error = []
training_size = [i*1000 for i in range(1, 10, 2)]

best_params = {'alpha': 1e-5, 'gamma': 1e-5, 'kernel': 'rbf'}

for size in training_size:
    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=1-size/10000, random_state=42)
    model = KernelRidge(**best_params)
    model.fit(X_train, y_train)
    preds = model.predict(X_test)
    error = mean_absolute_error(preds, y_test)
    normal_KRR_error.append(error)
    print(f"Training size: {size}. MAE: {error}.")

print(normal_KRR_error)

In [ ]:
x = training_size
y1 = gaussian_error # [2.443359366444566, 2.4407461385715497, 2.4368748118001204, 2.4359856271244116, 2.4406070370001216]
y2 = laplacian_error # [2.443359366444566, 2.4407461385715497, 2.4368748118001204, 2.4359856271244116, 2.4406070370001216]
y3 = normal_KRR_error # [0.04285921037085594, 0.033405950159750446, 0.02982421188819421, 0.027342219954064845, 0.025942363605852427]

plt.figure(figsize=(12, 8))
plt.plot(x, y1, label='CM - Gaussian', marker='^', linestyle='-', linewidth=3, markersize=10)
plt.plot(x, y2, label='CM - Laplacian', marker='^', linestyle='-', linewidth=3, markersize=10)
plt.plot(x, y3, label='CM - Euclidean distance', marker='^', linestyle='-', linewidth=3, markersize=10)

plt.title("Learning curve for commutator KRR with CM representation, atomization energy prediction")
plt.xlabel('Training size')
plt.ylabel('MAE [Ha]')
plt.legend()

plt.xscale('log', base=10)
plt.yscale('log', base=10)
xticks = training_size
yticks = [10, 1, 0.1, 0.01]
plt.xticks(xticks, labels=xticks)
plt.yticks(yticks, labels=yticks)

plt.savefig("Learning_curve.png")
plt.show()


## Distribution ##

In [ ]:
def euclidean_distance_1d(arr, batch_size=500):
    num_rows = arr.shape[0]
    distance_matrix = np.zeros((num_rows, num_rows))

    for i in range(0, num_rows, batch_size):
        end_idx_i = min(i + batch_size, num_rows)
        batch_i = arr[i:end_idx_i]

        for j in range(0, num_rows, batch_size):
            end_idx_j = min(j + batch_size, num_rows)
            batch_j = arr[j:end_idx_j]

            # Calculate squared Euclidean distances between rows in the current batches
            squared_distances = np.sum((batch_i[:, np.newaxis] - batch_j) ** 2, axis=2)

            # Take the square root to get the actual Euclidean distances
            distance_matrix[i:end_idx_i, j:end_idx_j] = np.sqrt(squared_distances)

    return distance_matrix


In [ ]:
vec_dist= euclidean_distance_1d(cm_flatten[:10000])
print(vec_dist.shape)


In [ ]:
flattened_vec_dist = vec_dist.flatten()
plt.hist(flattened_vec_dist, bins=50, alpha=0.7, color='blue', edgecolor='black')
plt.xlabel('Euclidean Distance')
plt.ylabel('Frequency')
plt.title('Distribution of Euclidean Distances between CM vectors')
plt.grid(True)
plt.savefig("Distribution of Euclidean Distances between CM vectors.png")
plt.show()

In [ ]:
def euclidean_distance_2d(arr, batch_size=1000):
    num_matrices = arr.shape[0]
    num_rows = arr.shape[1]
    distance_matrix = np.zeros((num_matrices, num_matrices))

    for i in range(0, num_matrices, batch_size):
        end_idx_i = min(i + batch_size, num_matrices)
        batch_i = arr[i:end_idx_i]

        for j in range(0, num_matrices, batch_size):
            end_idx_j = min(j + batch_size, num_matrices)
            batch_j = arr[j:end_idx_j]

            # Calculate squared Euclidean distances between matrices in the current batches
            squared_distances = np.sum((batch_i[:, np.newaxis] - batch_j) ** 2, axis=(2, 3))

            # Take the square root to get the actual Euclidean distances
            distance_matrix[i:end_idx_i, j:end_idx_j] = np.sqrt(squared_distances)

    return distance_matrix


In [ ]:
matrix_dist = euclidean_distance_2d(cm[:10000])
print(matrix_dist.shape)

In [ ]:
flattened_matrix_dist = matrix_dist.flatten()
plt.hist(flattened_matrix_dist, bins=50, alpha=0.7, color='blue', edgecolor='black')
plt.xlabel('Euclidean Distance')
plt.ylabel('Frequency')
plt.title('Distribution of Euclidean Distances between CM matrices')
plt.grid(True)
plt.savefig("Distribution of Euclidean Distances between CM matrices.png")
plt.show()

## Commutator Distribution ##

In [ ]:
def commutator_distance_matrix_batched(arr, batch_size=1000):
    num_matrices = arr.shape[0]
    num_rows = arr.shape[1]
    distance_matrix = np.zeros((num_matrices, num_matrices))

    for i in range(0, num_matrices, batch_size):
        end_idx_i = min(i + batch_size, num_matrices)
        batch_i = arr[i:end_idx_i]

        for j in range(0, num_matrices, batch_size):
            end_idx_j = min(j + batch_size, num_matrices)
            batch_j = arr[j:end_idx_j]

            # Compute the commutator between matrices in the current batches
            commutator = np.matmul(batch_i[:, np.newaxis], batch_j) - np.matmul(batch_j[:, np.newaxis], batch_i)
            
            # Calculate the Frobenius norm of the commutator
            commutator_norm = np.linalg.norm(commutator, ord='fro', axis=(2, 3))

            # Take the square root to get the actual commutator distances
            distance_matrix[i:end_idx_i, j:end_idx_j] = commutator_norm

    return distance_matrix

In [ ]:
# Compute the commutator distance matrix for matrices in batches of size 100
commutator_dist = commutator_distance_matrix_batched(cm[:10000])
print(commutator_dist.shape)


In [ ]:
# Flatten the distance matrix into a 1D array
flattened_commutator_dist = commutator_dist.flatten()

# Create a histogram of the flattened distances
plt.hist(flattened_commutator_dist, bins=50, alpha=0.7, color='blue', edgecolor='black')
plt.xlabel('Commutator Distance')
plt.ylabel('Frequency')
plt.title('Distribution of Commutator Distances between Pairwise Matrices')
plt.grid(True)
# plt.savefig('Distribution of Commutator Distances between Pairwise Matrices')
plt.show()

In [ ]:
unique_sorted = np.sort(np.unique(flattened_commutator_dist))
print(unique_sorted[1])

In [ ]:
zero_mask = (flattened_commutator_dist == 0)
num_zeros = np.count_nonzero(zero_mask)
small_mask = (flattened_commutator_dist < 10)
num_small = np.count_nonzero(small_mask)
print(num_zeros)
print(num_small)

## Commutator With Eigval ##

In [ ]:
def process_batch_commutator_eigval(args):
    batch_i, batch_j = args
    commutator = np.matmul(batch_i[:, np.newaxis], batch_j) - np.matmul(batch_j[:, np.newaxis], batch_i)
    commutator_norm = np.linalg.norm(commutator, ord='fro', axis=(2, 3))
    eigenvalue_diff_norm = np.linalg.norm(np.linalg.eigvals(batch_i) - np.linalg.eigvals(batch_j), ord=2, axis=1)
    combined_norms = commutator_norm + eigenvalue_diff_norm
    return combined_norms

def commutator_distance_with_eigval_parallel(arr, batch_size=1000, num_processes=4):
    num_matrices = arr.shape[0]
    distance_matrix = np.zeros((num_matrices, num_matrices))

    if num_processes is None:
        num_processes = multiprocessing.cpu_count()

    pool = multiprocessing.Pool(processes=num_processes)

    for i in range(0, num_matrices, batch_size):
        end_idx_i = min(i + batch_size, num_matrices)
        batch_i = arr[i:end_idx_i]

        args_list = []
        for j in range(0, num_matrices, batch_size):
            end_idx_j = min(j + batch_size, num_matrices)
            batch_j = arr[j:end_idx_j]
            args_list.append((batch_i, batch_j))

        results = pool.map(process_batch_commutator_eigval, args_list)

        for j, result in enumerate(results):
            end_idx_j = min(j + batch_size, num_matrices)
            distance_matrix[i:end_idx_i, j:end_idx_j] = result

    pool.close()
    pool.join()

    return distance_matrix


In [ ]:
def commutator_distance_with_eigval(arr, batch_size=1000):
    num_matrices = arr.shape[0]
    distance_matrix = np.zeros((num_matrices, num_matrices))

    for i in range(0, num_matrices, batch_size):
        end_idx_i = min(i + batch_size, num_matrices)
        batch_i = arr[i:end_idx_i]

        for j in range(0, num_matrices, batch_size):
            end_idx_j = min(j + batch_size, num_matrices)
            batch_j = arr[j:end_idx_j]

            # Compute the commutator between matrices in the current batches
            commutator = np.matmul(batch_i[:, np.newaxis], batch_j) - np.matmul(batch_j[:, np.newaxis], batch_i)
            
            # Calculate the Frobenius norm of the commutator
            commutator_norm = np.linalg.norm(commutator, ord='fro', axis=(2, 3))

            # Calculate the L2 norm of eigenvalue differences
            eigenvalue_diff_norm = np.linalg.norm(np.linalg.eigvals(batch_i) - np.linalg.eigvals(batch_j), ord=2, axis=1)

            # Combine the norms element-wise
            combined_norms = commutator_norm + eigenvalue_diff_norm

            # Update the distance matrix
            distance_matrix[i:end_idx_i, j:end_idx_j] = combined_norms

    return distance_matrix

In [ ]:
def eigval_distance(arr, batch_size=1000):
    num_matrices = arr.shape[0]
    distance_matrix = np.zeros((num_matrices, num_matrices))

    for i in range(0, num_matrices, batch_size):
        end_idx_i = min(i + batch_size, num_matrices)
        batch_i = arr[i:end_idx_i]

        for j in range(0, num_matrices, batch_size):
            end_idx_j = min(j + batch_size, num_matrices)
            batch_j = arr[j:end_idx_j]

            # Calculate the L2 norm of eigenvalue differences
            eigenvalue_diff_norm = np.linalg.norm(np.linalg.eigvals(batch_i) - np.linalg.eigvals(batch_j), ord=2, axis=1) ** 2

            # Update the distance matrix
            distance_matrix[i:end_idx_i, j:end_idx_j] = eigenvalue_diff_norm

    return distance_matrix


In [ ]:
eigval_dist = eigval_distance(cm[:10000])
print(eigval_dist.shape)

In [ ]:
commutator_eigval_dist = eigval_dist + commutator_dist
print(commutator_eigval_dist.shape)

In [ ]:
# Flatten the distance matrix into a 1D array
flattened_commutator_eigval = commutator_eigval_dist.flatten()
flattened_eigval_dist = eigval_dist.flatten()

# Create a histogram of the flattened distances
fig, axes = plt.subplots(2, 2, figsize=(12, 9))

axes[0, 0].hist(flattened_commutator_dist, bins=50, alpha=0.7, color='red', edgecolor='black')
axes[0, 0].set_title('Commutator Norm')
axes[0, 0].set_xlabel('Norm')
axes[0, 0].set_ylabel('Frequency')

axes[0, 1].hist(flattened_commutator_eigval, bins=50, alpha=0.7, color='blue', edgecolor='black')
axes[0, 1].set_title('Commutator + Eigenvalue Norm')
axes[0, 1].set_xlabel('Norm')
axes[0, 1].set_ylabel('Frequency')

axes[1, 0].hist(flattened_eigval_dist, bins=50, alpha=0.7, color='green', edgecolor='black')
axes[1, 0].set_title('Eigenvalue Norm')
axes[1, 0].set_xlabel('Norm')
axes[1, 0].set_ylabel('Frequency')

axes[1, 1].hist(flattened_commutator_eigval, bins=50, alpha=0.7, color='blue', edgecolor='black', label="Commutator + Eigenvalue norm")
axes[1, 1].hist(flattened_commutator_dist, bins=50, alpha=0.7, color='red', edgecolor='black', label="Commutator norm")
axes[1, 1].set_title('Comparison')
axes[1, 1].set_xlabel('Norm')
axes[1, 1].set_ylabel('Frequency')
axes[1, 1].legend()

plt.tight_layout()
plt.suptitle('Comparing Distribution of Commutator and Eigenvalue Distance between Pairwise Matrices')
plt.subplots_adjust(top=0.90, hspace=0.2)
plt.grid(True)
plt.savefig('Comparing Distribution of Commutator and Eigenvalue Distance between Pairwise Matrices')
plt.show()

In [ ]:
# Flatten the distance matrix into a 1D array
flattened_eigval_dist = eigval_dist.flatten()

# Create a histogram of the flattened distances
plt.hist(flattened_eigval_dist, bins=50, alpha=0.7, color='blue', edgecolor='black')
plt.xlabel('Eigval Distance')
plt.ylabel('Frequency')
plt.title('Distribution of Eigenvalue Distance between Pairwise Matrices')
plt.grid(True)
plt.savefig('Distribution of Eigenvalue Distance between Pairwise Matrices')
plt.show()

In [ ]:
zero_mask = (flattened_commutator_dist == 0)
num_zeros = np.count_nonzero(zero_mask)
zero_mask_w_eig = (flattened_commutator_eigval == 0)
num_zeros_w_eig = np.count_nonzero(zero_mask_w_eig)
print(num_zeros)
print(num_zeros_w_eig)

## Sanity Check ##

In [ ]:
zero_indices = np.argwhere(commutator_eigval_dist == 0)
print(zero_indices[:10])

In [ ]:
for i in range(10):
    index1, index2 = zero_indices[i][0], zero_indices[i][1]
    print(f"duplicate {i+1}:")
    print(elements[index1])
    print(elements[index2])
    print()